In [1]:
%pip install opendatasets --upgrade --quiet

import opendatasets as od
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


C:\Users\igors\AppData\Local\Temp\ipykernel_12728\3003167959.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# insert ypu kaggle  username and key
od.download('https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small',data_dir="dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading fashion-product-images-small.zip to ../dataset\fashion-product-images-small


100%|██████████| 565M/565M [01:56<00:00, 5.09MB/s] 
